In [ ]:
%reload_ext XXX.mumble

In [ ]:

    get_ipython().ast_node_interactivity='last_expr_or_assign'

In [ ]:

    from poser import *
    from notebook.utils import url_path_join as ujoin
    from importlib import reload
    import requests
    import requests_cache
    requests_cache.install_cache("election")

In [ ]:
    ---
    urls:
        prefix: http://results.enr.clarityelections.com/GA/
        reports: http://results.enr.clarityelections.com/GA/63991/184321/en/reports.html
        ga: http://results.enr.clarityelections.com/GA/
        elections: http://results.enr.clarityelections.com/GA/elections.json
        county: http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html
        franklin: http://results.enr.clarityelections.com/GA/Franklin/64051/
        franklin_redir: http://results.enr.clarityelections.com/GA/Franklin/64051/183122/en/summary.html
        json: http://results.enr.clarityelections.com/GA/63991/184321/json/status.json
        select: http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html
        wiki: https://en.wikipedia.org/wiki/List_of_counties_in_Georgia_(U.S._state)

In [ ]:

    read_html, get =(a/cache)((a.pandas.read_html(), a.requests.get()))
    get = requests.get

In [ ]:
    num_counties = (a[read_html].drop(2).first().len()[x-1])(urls['wiki'])

In [ ]:
# A List of All of the Counties

    soup = a[get][x.text].bs4.BeautifulSoup(features='lxml')(urls['county'])
    county = (a.map(x.attrs['value']).list())(soup.select('table tr li a'));
    assert len(county) is num_counties, """The number of rows is not the number of counties."""

In [ ]:
# The County Event ID

    event_ids = a.map(a.str.split('/').filter(bool).take(2)).dict()(county);
    a.dict.items().take(3).dict()(event_ids)

In [ ]:
    def construct_context_version(name='Appling', sleep=1):
        latest_version = a[[a.identity(), event_ids.__getitem__]][
            [urls["prefix"]], a.identity(), ['current_ver.txt']
        ].concat()['/'.join][get][x.text]

        constructed = ujoin(urls["prefix"], name, event_ids[name], latest_version(name))
        constructed = ujoin(constructed , 'json/{}.json').format
        return a.map(constructed).map((
            a.str.rsplit('/', 1).last().str.replace('.json', ''), 
            a[get].methodcaller('json'))).dict()(('sum', 'details', 'status'))

In [ ]:
    ---
    labels:
        CH: Candidate
        C: Office
        V: Votes
        TV: Total Voters
        BC: Ballot Cast

In [ ]:
    def tidy_precinct(s):
        return a.zip().dict().valmap(
            a.zip(s.Candidate).dict()
        ).pandas.DataFrame()(s.P, s.V)

In [ ]:
    dfs = {}
    for i, county in enumerate(event_ids):
        print(i, county)
        objects = construct_context_version(county)
        
        df = (a.pandas.DataFrame())(objects['sum']['Contests'])
        
        # assert df.V.apply(sum).lt(df.TV).all()
        df = df[list(labels)].rename(columns=labels)
        df = (a.pandas.DataFrame()[df.join])(objects['details']['Contests'])

        dfs[county] = a.pandas.concat()({index: tidy_precinct(row)
            for index, row in df.set_index('Office').iterrows() 
            if (a**list)(row.P) or not a.pandas.np.isnan()(row.P)})

In [ ]:
    tidy = a.pandas.concat()(dfs)

    tidy = tidy.swaplevel(0,1).stack().pipe(
        lambda df: a.pandas.Series()(df.values, df.index.rename(('contest', 'county', 'candidate', 'precinct')))
    ).rename('votes')

In [ ]:
    tidy.to_csv('all_contests_precinct.csv')
    tidy.loc['President of the United States'].to_csv('president_precinct.csv')

> I want election **results** by **precinct** in **Georgia** for the **2016 presidential election**. The problem is that results are broken down by county in details.xls on the Georgia Secretary of State's website. However, precinct-level details are available for each of Georgia's 159 counties 

![](./state_list.png)

in individual reports (also titled details.xls). 

![](./county_example.png)

In [ ]:
    the.pandas.DataFrame()(tidy)

In [ ]:

    import Maps
    reload(Maps)

In [ ]:

    vote_county_precincts = (
        tidy['President of the United States']
        .reset_index()[["county", "precinct", "candidate", "votes"]]
        .rename({"county": "COUNTY_NAM", "precinct": "PRECINCT_N"}, axis="columns")
        .sort_values(['COUNTY_NAM', 'PRECINCT_N'])
        .drop_duplicates()
        .dropna()
    )
    vote_county_precincts['COUNTY_NAM'] = vote_county_precincts['COUNTY_NAM'].str.upper()
    vote_county_precincts['PRECINCT_N'] = vote_county_precincts['PRECINCT_N'].str.upper()
    vote_county_precincts = vote_county_precincts.set_index(['COUNTY_NAM', 'PRECINCT_N'])

In [ ]:

    map_county_precincts = (Maps.precincts_2014[['PRECINCT_N', 'COUNTY_NAM', 'geometry']]
                            .sort_values(['COUNTY_NAM', 'PRECINCT_N'])
                            .dropna())
    map_county_precincts['PRECINCT_N'] = map_county_precincts['PRECINCT_N'].apply(str)
    map_county_precincts = map_county_precincts.set_index(['COUNTY_NAM', 'PRECINCT_N'])

In [ ]:

    joined = map_county_precincts.join(vote_county_precincts, how='outer').dropna()

In [ ]:

    trump = joined['candidate'].str.contains('TRUMP')
    clinton = joined['candidate'].str.contains('CLINTON');

In [ ]:

    red = joined[(joined[trump]['votes'] > joined[clinton]['votes']) & trump]
    red['color'] = 1
    red.size

In [ ]:

    blue = joined[(joined[trump]['votes'] < joined[clinton]['votes']) & clinton]
    blue['color'] = 0
    blue.size

In [ ]:

    red_blue = the.pandas.concat()([blue, red])

In [ ]:
    red_blue.plot(column='color', cmap='bwr');